In [ ]:
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import Video
import xml.etree.ElementTree as ET

# Demo 2: Creating an Application-Specific Data Layout
In this demonstration, we will go over how to create an XML file that describes the data layout specifically designed for 2D DFT. PUM-friendly data layout is a critical part of executing an application on any PUM system. In Demo 1, we have gone through how to execute a program in MASTODON and gather statistics like simulated execution time. Combining that with Demo 2, MASTODON can provide a testbed for research in automatic data layout engines, compile-time routines and infrastructures that can aid this process.

### Step 1: Generating the Input Data

There are 2 types of input neccessary for the 2D DFT application: the DFT matrix and the input image

Things to note:
- We will use a 64x64 input image
- We will perform DFT on a node with 8 lanes, 64 register files per lane, 128 registers per register file, 64 elements per register

In [ ]:
def create_dft_matrix(N):
    PI = np.pi
    real_matrix = np.zeros((N, N), dtype=np.int32)
    imag_matrix = np.zeros((N, N), dtype=np.int32)
    
    for k in range(N):
        for n in range(N):
            angle = -2 * PI * k * n / N
            scale = 512
            
            realval = int(np.cos(angle) * scale) + scale
            imagval = int(np.sin(angle) * scale) + scale
            
            real_matrix[k][n] = realval
            imag_matrix[k][n] = imagval
    
    return real_matrix, imag_matrix

def load_image_matrix(N):
    with Image.open("./img/ichiro.jpg") as img:
    # Resize image to 64x64 pixels
        img_resized = img.resize((N, N))
        img_gray = img_resized.convert("L")
    pixels = np.asarray(list(img_gray.getdata()))
    return pixels.reshape((N, N))

N = 64
real_matrix, imag_matrix = create_dft_matrix(N)
input_image = load_image_matrix(N)
plt.subplot(1, 2, 1)
plt.imshow(real_matrix, cmap='gray', interpolation='none')
plt.subplot(1, 2, 2)
plt.imshow(input_image, cmap = 'gray',interpolation='none')

### Step 2: Populating the 4D Input Array
A location of an element in a node can be dereferenced using its Lane, Vector Register File, Vector Register Index, and Element Index. Hence, we create a 4-D array called `data_in` and populate the input data (with user-defined layout)

In [ ]:
L = 8    # Number of lanes in a node
F = 64   # Number of reg. files in a lane
C = 128  # Number of vector registers in a reg. file
R = 64   # Nimber of element in a vector register

# Initialize the 4D list with zeros
data_in = [[[[0 for _ in range(R)] for _ in range(C)] for _ in range(F)] for _ in range(L)]

# Populate real dft matrix in regfile 0 - 63, for register 0 - 63, lane 0
for f in range(64):
    for c in range(64):
        for r in range(R):
            data_in[0][f][c][r] = real_matrix[c][r]

# Populate imaginary dft matrix in regfile 0 - 63, for register 0 - 63, lane 1
for f in range(64):
    for c in range(64):
        for r in range(R):
            data_in[1][f][c][r] = imag_matrix[c][r]

# Populate the input signal in regfile 0 - 63, for register 64 - 127, lane 0 and 1
for l in range(2):
    for f in range(64):
        for c in range(64):
            for r in range(R):
                data_in[l][f][c + 64][r] = input_image[f][c]

# Populate the transposed matrix in regfile 0 - 63, for register 64 - 127, lane 2 and 3
for f in range(64):
    for c in range(64):
        for r in range(R):
            data_in[2][f][c + 64][r] = real_matrix.T[f][c]
            data_in[3][f][c + 64][r] = imag_matrix.T[f][c]

### Step 3: Visualizing the Node's Content
Things to note:
- Based on the mapping above, we should see the DFT matrix occupying the first 64 registers of every register files
- One column of the input image is transpoised and replicated to every row of register 64 - 127 of the same register files

In [ ]:
plt.subplot(2, 2, 1)
plt.title("Lane 0, Reg. File 0")
plt.imshow(np.array(data_in[0][0]).T, cmap='gray', interpolation='none')
plt.subplot(2, 2, 2)
plt.title("Lane 0, Reg. File 8")
plt.imshow(np.array(data_in[0][8]).T, cmap='gray', interpolation='none')
plt.subplot(2, 2, 3)
plt.title("Lane 0, Reg. File 16")
plt.imshow(np.array(data_in[0][16]).T, cmap='gray', interpolation='none')
plt.subplot(2, 2, 4)
plt.title("Lane 0, Reg. File 32")
plt.imshow(np.array(data_in[0][32]).T, cmap='gray', interpolation='none')

### Step 4: Creating the XML MASTODON Input from the 4-D Array
Things to note:
- Without loss of generality, you can create the XML file and the 4-D array in any language you want
- As long as the generated XML file is in the correct format, it can be fed to MASTODON

In [ ]:
def create_xml(data_in, file_addr, num_lane, num_regfile, num_col, num_row):
    root = ET.Element("root")
    node = ET.SubElement(root, "in")
    
    for l in range(num_lane):
        lane_node = ET.SubElement(node, f"L{l}")
        for f in range(num_regfile):
            regfile_node = ET.SubElement(lane_node, f"RF{f}")
            for c in range(num_col):
                data = " ".join(str(data_in[l][f][c][r]) for r in range(num_row))
                ET.SubElement(regfile_node, f"C{c}").text = data
    
    tree = ET.ElementTree(root)
    tree.write(file_addr, encoding='utf-8', xml_declaration=True)

create_xml(data_in, "DFT.xml", L, F, C, R)

### Step 5: Running the DFT application
The DFT applications will take around 30-40 minutes to run. For your convenience, we have created a GoogleTest suite so that you can run different tests (including DFT) yourself. To run the DFT example, under the `gtest` folder, do `./gtest --gtest_filter=*DFT2D`. You can see the test code under `./gtest/suite/SingleNode_Microkernels.cpp`. After the test is complete, you can run the following code to view the output of DFT.

In [ ]:
def read_text_file_to_array(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Convert lines of text into a 2D list of integers
    data = [list(map(int, line.split())) for line in lines]
    return np.abs(np.array(np.array(data)[1:]).T[1:])

def normalize_pixel_values(array):
    # Normalize pixel values to range [0, 255] if necessary
    min_val = np.min(array)
    max_val = np.max(array)
    if min_val < 0 or max_val > 255:
        array = 255 * (array - min_val) / (max_val - min_val)
    return array.astype(np.uint8)
    
def convert_negative(array):
    output = []
    for row in array:
        converted_row = []
        for elem in row:
            if (elem > 4294967296):
                elem = ~elem + 1
            converted_row.append(elem)
        output.append(converted_row)
    return np.asarray(output)
    
def display_image(array):
    plt.imshow(array, cmap='gray', vmin=0, vmax=255)
    plt.axis('off')  # Turn off axis numbers and ticks
    plt.show()

# Example usage
filename = '../../gtest/suite/dft-isca/output.txt'  # Replace with your text file name
pixel_array = read_text_file_to_array(filename)
normalized_array = normalize_pixel_values(pixel_array)
display_image(normalized_array)